# State Probability All Probabilities PS Example

This example demonstrates how to compute all state probabilities in a processor sharing (PS) queueing network.

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.nc.*
import jline.solvers.mva.*

In [ ]:
// Set verbose level
GlobalConstants.setVerbose(VerboseLevel.STD)

// Create network with PS queues
val model = Network("myModel")

// Block 1: nodes (PS scheduling)
val node1 = Delay(model, "Delay")
val node2 = Queue(model, "Queue1", SchedStrategy.PS)
val node3 = Queue(model, "Queue2", SchedStrategy.PS)
node2.setNumberOfServers(1)
node3.setNumberOfServers(1)

In [ ]:
// Block 2: classes
val jobclass1 = ClosedClass(model, "Class1", 2, node1, 0)
val jobclass2 = ClosedClass(model, "Class2", 2, node1, 0)

In [ ]:
// Set service times
node1.setService(jobclass1, Exp.fitMean(1.0))
node1.setService(jobclass2, Exp.fitMean(1.0))

node2.setService(jobclass1, Exp.fitMean(0.6))
node2.setService(jobclass2, Exp.fitMean(0.4))

node3.setService(jobclass1, Exp.fitMean(0.8))
node3.setService(jobclass2, Exp.fitMean(1.2))

In [ ]:
// Block 3: routing
val P = model.initRoutingMatrix()

// Cyclic routing
P.set(jobclass1, jobclass1, node1, node2, 1.0)
P.set(jobclass1, jobclass1, node2, node3, 1.0)
P.set(jobclass1, jobclass1, node3, node1, 1.0)

P.set(jobclass2, jobclass2, node1, node2, 1.0)
P.set(jobclass2, jobclass2, node2, node3, 1.0)
P.set(jobclass2, jobclass2, node3, node1, 1.0)

model.link(P)

In [ ]:
// Solve with CTMC to get all state probabilities
println("Solving with CTMC for all state probabilities (PS network):")
try {
    val solverCtmc = CTMC(model)
    val avgTable = solverCtmc.avgTable
    println("CTMC Results:")
    avgTable.print()
    
    // Get all state probabilities
    println("\nTesting all state probabilities methods:")
    try {
        val allProbs = solverCtmc.probSysAggr
        println("System aggregated probabilities computed successfully")
        println("Number of states: ${allProbs.size}")
        
        // Test node-specific probabilities
        node2.setState(intArrayOf(1, 1)) // Specific state [1,1]
        val nodeProb = solverCtmc.getProbAggr(node2)
        println("Probability of state [1,1] at Queue1: $nodeProb")
    } catch (e: Exception) {
        println("State probability methods not available: ${e.message}")
    }
    
} catch (e: Exception) {
    println("CTMC solver error: ${e.message}")
}

In [ ]:
// Compare with MVA solution (exact for PS networks)
println("\nComparing with MVA solver:")
try {
    val solverMva = MVA(model)
    val avgTableMva = solverMva.avgTable
    println("MVA Results:")
    avgTableMva.print()
} catch (e: Exception) {
    println("MVA solver error: ${e.message}")
}